In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
from remote_sensing_tools.raster_base import RasterBase, QueryParams, LoadParams
from remote_sensing_tools.stac_configuration import de_africa_stac_config

## Set up query and load params

In [ ]:
query_params = QueryParams(
    bbox=(37.76, 12.49, 37.77, 12.50),
    start_date="2020-11-01",
    end_date="2020-12-01",
)

load_params = LoadParams(
    crs="EPSG:6933", resolution=20, bands=("red", "green", "blue", "nir", "scl")
)

## Load using class method

In [ ]:
stac_raster = RasterBase.from_stac_query(
    config=de_africa_stac_config,
    collections=["s2_l2a"],
    query_params=query_params,
    load_params=load_params,
)

## Inspect attributes of raster

In [ ]:
stac_raster.data

In [ ]:
stac_raster.masks

## Demonstrate how to access various required metadata

In [ ]:
# CRS
epsg_number = stac_raster.data.odc.crs.epsg
print(f"ESPG number: {epsg_number}")
crs = stac_raster.data.odc.crs.to_wkt()
print(f"WKT CRS: {crs} \n")

# Bounds
extent = stac_raster.data.odc.geobox.extent.boundingbox.bbox
print(f"Data CRS Bounds: {extent}")
bounds = stac_raster.data.odc.map_bounds()
print(f"Lat Lon CRS Bounds: {bounds} \n")

# Resolution
resolution = stac_raster.data.odc.geobox.resolution
print(f"Resolution: {resolution}")
print(f"Resolution in x: {resolution.x}")
print(f"Resolution in y: {resolution.y} \n")

# Shape
height = stac_raster.data.odc.geobox.height
width = stac_raster.data.odc.geobox.width
print(f"Shape: {height, width} \n")

# Transform
transform = stac_raster.data.odc.geobox.transform
print(f"Transform: {transform} \n")

# Band level info
blue_nodata_value = stac_raster.data.blue.nodata
print(f"Blue nodata value: {blue_nodata_value}")
blue_type = stac_raster.data.blue.dtype
print(f"Blue type: {blue_type}")